# Applying UBIX to trained models

This notebook describes both how to predict using already trained models and how to insert UBIX during the prediction of these trained models.

In [ ]:
# !pip3 install -e .

In [ ]:
import os

os.environ["WANDB_SILENT"] = "true"
os.environ['WANDB_NOTEBOOK_NAME'] = 'Applying UBIX to trained models'
os.environ["WANDB_CONSOLE"] = "off" 
os.environ["WANDB_MODE"] = "dryrun"

from ubix.ubix_class import UBIX
from ubix.utils import set_random_seeds
from ubix.uncertainty.deep_ensemble import deep_ensemble
from ubix.predict import Predict

from ubix.metrics import quadratic_weighted_kappa

set_random_seeds(seed=0)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Model prediction (without UBIX)

Generally, we use the workflow of first predicting with already trained models, without UBIX. This saves the predictions (including the instance-level predictions), which will be used later by our UBIX modules. In this way, do not have to run the instance-level classifier again each time we want to use different UBIX variant (e.g., UBIX<sub>hard</sub> or UBIX<sub>soft</sub>, but also with different uncerainty estimation measures and techniques).

Let's load in an already trained model. In this example, we use a model pretrained by us (the weights will be loaded from a publicly available URL described in `ubix/public_models_urls.json`). However, you can also simply use a model name of a model that you trained.

In [ ]:
model_name = "pretrained-maxpooling"

And here, provide the path to the data that you would like to predict. How to prepare your data is described in the `README.md` file of this repository.

In [ ]:
data_root = 'ubix-dummy-data'

Now, we perform the prediction of the trained deep ensemble, for both the validation and test set. Here, replace `data_root` the path to the data that you would like to predict.

In [ ]:
for predict_set in ['test']:  # If you only have test data, use ['test'], otherwise use ['val', 'test']
    deep_ensemble(
        model_base_name=model_name,
        model_type="best_quadratic_weighted_kappa",
        n_its=5,
        load_from_file=False,
        predict_set=predict_set,
        change_config={
            'data_root': data_root,
            'num_workers': 0
        }
    )

In [ ]:
predictor = Predict(
    model_name=model_name,
    model_type="best_quadratic_weighted_kappa",
    do_not_load_model=True,
    change_config={
        'data_root': data_root,
        'num_workers': 0
    }
)

In [ ]:
predictions_postfix = "_deep_ensemble_5_mean"
predictions = predictor.predict_test_set(load_from_file=True, predictions_postfix=predictions_postfix)

Prediction outcomes are now saved in the `predictions` variable.

For example, `predictions['y_pred']` contains logit outputs:

In [ ]:
predictions['y_pred']

And the predicted classes can be determined as follows:

In [ ]:
predictions['y_pred'].argmax(-1)

Now you can calculate metrics such as the quadratic weighted kappa score:

In [ ]:
quadratic_weighted_kappa(predictions['y_true'], predictions['y_pred'])

## Model prediction with UBIX

You can either use the UBIX parameters that were determined with our validation set, or you can optimize UBIX using your own dataset.

We first show how to use our UBIX parameters (optimized using the dataset H<sub>val</sub>, as described in the paper).

In [ ]:
ubix_type = "hard"  # other option: "soft"
uncertainty_measure = "ordinal_entropy"  # other options include: "entropy", "max_logit_max0th_confidence", "confidence", "variance", "ordinal_variance" 

save_path = f"./results/models/{model_name}/ubix_params_hval_{ubix_type}{predictions_postfix}_{uncertainty_measure}.p"

ubix_module = UBIX(ubix_type=ubix_type, uncertainty_measure=uncertainty_measure)
ubix_module.load_ubix_parameters(save_path);

In [ ]:
y_pred_ubix = ubix_module.predict(
    predictor, predict_set="test", predictions_postfix=predictions_postfix
)

In [ ]:
quadratic_weighted_kappa(predictions['y_true'], y_pred_ubix)

To optimize with a new dataset, use the `UBIX.optimize_ubix_parameters` function as follows:

In [ ]:
optimization_predictor = Predict(
    model_name=model_name,
    model_type="best_quadratic_weighted_kappa",
    do_not_load_model=True,
)
ubix_module.optimize_ubix_parameters(
    optimization_predictor,
    predict_set="val",
    predictions_postfix=predictions_postfix,
)

Optimized parameters can be saved as follows:

In [ ]:
ubix_module.save_ubix_parameters(f"./results/models/{model_name}/ubix_params_custom.p")